## Modelado de Temas con LDA

### Instalación de Librerías

In [1]:
!pip install pandas gensim spacy nltk

DEPRECATION: Loading egg at c:\programdata\miniconda3\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


### Implementación LDA

In [4]:
import json
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS as GENSIM_STOP
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel, Phrases
from gensim.models.phrases import Phraser
from tqdm import tqdm

# -------------------------------------------------
# 1. Descargas necesarias (solo la primera vez)
# -------------------------------------------------
nltk.download('stopwords', quiet=True)

# Definir stopwords en español y stemmer
nltk_stop = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')


# -------------------------------------------------
# 2. Función de preprocesado (tokenización + stemming)
# -------------------------------------------------
def preprocess(texto: str) -> list[str]:
    """
    1) Convierte a minúsculas y tokeniza quitando puntuación y acentos (deacc=True).
    2) Filtra tokens de longitud <= 3.
    3) Elimina stopwords de gensim y NLTK.
    4) Aplica stemming en español.
    Devuelve la lista de stems resultantes.
    """
    tokens = simple_preprocess(texto, deacc=True)
    tokens = [t for t in tokens if len(t) > 3 and t not in GENSIM_STOP and t not in nltk_stop]
    stems = [stemmer.stem(t) for t in tokens]
    return stems


# -------------------------------------------------
# 3. (Opcional) Generación de bigramas
# -------------------------------------------------
def aplicar_bigramas(
        documentos: list[list[str]],
        min_count: int = 5,
        threshold: float = 100.0
) -> tuple[list[list[str]], Phraser]:
    """
    1) Entrena un modelo de bigramas sobre los documentos tokenizados.
    2) Devuelve los documentos enriquecidos con bigramas (uni_bi) y el modelo Phraser.
    Parámetros:
      - min_count: número mínimo de apariciones conjuntas para considerar bigrama.
      - threshold: umbral de formación de bigramas (valor alto = más conservador).
    """
    bigram = Phrases(documentos, min_count=min_count, threshold=threshold, progress_per=10000)
    bigram_mod = Phraser(bigram)
    documentos_bi = [bigram_mod[doc] for doc in documentos]
    return documentos_bi, bigram_mod


# -------------------------------------------------
# 4. Construcción de diccionario y corpus (BoW)
# -------------------------------------------------
def construir_diccionario_corpus(
        textos_tokenizados: list[list[str]],
        no_below: int = 5,
        no_above: float = 0.5
) -> tuple[corpora.Dictionary, list]:
    """
    1) Crea un Dictionary a partir de la lista de documentos (lista de tokens).
    2) Filtra palabras muy raras (no_below) o demasiado frecuentes (no_above).
    3) Construye el corpus en formato BoW (lista de tuplas (id_token, frecuencia)).
    Devuelve el diccionario filtrado y el corpus BoW.
    """
    diccionario = corpora.Dictionary(textos_tokenizados)
    diccionario.filter_extremes(no_below=no_below, no_above=no_above)
    corpus = [diccionario.doc2bow(doc) for doc in textos_tokenizados]
    return diccionario, corpus


# -------------------------------------------------
# 5. Entrenamiento y evaluación del modelo LDA
# -------------------------------------------------
def entrenar_lda(
        corpus: list,
        diccionario: corpora.Dictionary,
        num_topics: int = 7,
        passes: int = 10,
        random_state: int = 42,
        alpha: str = 'auto'
) -> LdaModel:
    """
    Entrena un modelo LDA con los parámetros indicados y devuelve el objeto LdaModel.
    """
    lda_model = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=random_state,
        passes=passes,
        alpha=alpha,
        per_word_topics=True
    )
    return lda_model


def evaluar_lda(
        lda_model: LdaModel,
        corpus: list,
        textos_tokenizados: list[list[str]],
        diccionario: corpora.Dictionary
) -> tuple[float, float]:
    """
    1) Calcula la perplejidad del modelo sobre el corpus.
    2) Calcula la coherencia (c_v) usando CoherenceModel.
    Devuelve (perplejidad, coherencia).
    """
    perplejidad = lda_model.log_perplexity(corpus)
    coh_model = CoherenceModel(
        model=lda_model,
        texts=textos_tokenizados,
        dictionary=diccionario,
        coherence='c_v'
    )
    coherencia = coh_model.get_coherence()
    return perplejidad, coherencia


# -------------------------------------------------
# 6. Asignación de tópico dominante a cada documento
# -------------------------------------------------
def asignar_topic_principal(lda_model: LdaModel, corpus: list) -> list[int]:
    """
    Para cada documento en formato BoW, obtiene la lista de tópicos con sus probabilidades
    y retorna el ID del tópico con mayor probabilidad (dominante).
    """
    topicos_principales = []
    for bow in corpus:
        distribucion = lda_model.get_document_topics(bow, minimum_probability=0.0)
        topico_dom = max(distribucion, key=lambda x: x[1])[0]
        topicos_principales.append(topico_dom)
    return topicos_principales


# -------------------------------------------------
# 7. Leer JSON y preparar DataFrame con 'description' y 'category'
# -------------------------------------------------
def cargar_descriptions_desde_json(ruta_json: str) -> pd.DataFrame:
    """
    Lee un archivo JSON con estructura de lista de objetos:
    [
      {
        "title": "...",
        "category": "...",
        "summit": "...",
        "description": "...",
        "date": "...",
        "autor": "...",
        "tags": "['Seguro Social', 'Estados Unidos']",
        "url": "..."
      },
      ...
    ]
    Extrae las columnas 'description' y 'category', elimina filas con description vacía.
    Devuelve un DataFrame con dichas columnas.
    """
    # Cargar JSON en un DataFrame
    df = pd.read_json(ruta_json, orient='records', encoding='utf-8')
    # Filtrar filas donde 'description' exista y no esté vacío
    df = df[df['description'].notna() & (df['description'].str.strip() != "")].reset_index(drop=True)
    return df[['description', 'category']]

### Identificación de 7 tópicos mediante LDA y evaluación del modelo

In [8]:
# -------------------------------------------------
# 8. Función principal: pipeline completo adaptado a JSON
# Parámetros ajustables
RUTA_JSON = "gestionspider5.json"        # Nombre del JSON a leer
NUM_TOPICS = 7                     # Número de tópicos para LDA
PASSES = 10                        # Número de pasadas (iteraciones) en el entrenamiento LDA
NO_BELOW = 5                       # Filtrar tokens que aparezcan en menos de NO_BELOW documentos
NO_ABOVE = 0.5                     # Filtrar tokens que aparezcan en más del 50% de documentos
BIGRAM = True                      # Si quieres incluir n-gramas (True o False)
BIGRAM_MIN_COUNT = 5               # Mínimo de apariciones para formar un bigrama
BIGRAM_THRESHOLD = 100.0           # Umbral de formación de bigramas

# 1) Leer el JSON y extraer 'description' y 'category'
df = cargar_descriptions_desde_json(RUTA_JSON)
print(f"Cargados {len(df)} registros desde '{RUTA_JSON}'.\n")

# 2) Preprocesar los textos en 'description'
textos_raw = df['description'].astype(str).tolist()
documentos_tokenizados = []
print("Preprocesando descripciones (tokenización + stemming)...")
for doc in tqdm(textos_raw, desc="Progreso preprocesado"):
    documentos_tokenizados.append(preprocess(doc))

# 3) (Opcional) Generar e incluir bigramas
if BIGRAM:
    print("\nDetectando bigramas en el corpus...")
    documentos_bi, modelo_bigram = aplicar_bigramas(
        documentos_tokenizados,
        min_count=BIGRAM_MIN_COUNT,
        threshold=BIGRAM_THRESHOLD
    )
    textos_finales = documentos_bi
else:
    textos_finales = documentos_tokenizados

# 4) Construir diccionario y corpus BoW
print("\nConstruyendo diccionario y corpus BoW...")
diccionario, corpus = construir_diccionario_corpus(
    textos_tokenizados=textos_finales,
    no_below=NO_BELOW,
    no_above=NO_ABOVE
)
print(f"Diccionario creado: {len(diccionario)} tokens únicos.\n")

# 5) Entrenar modelo LDA
print(f"Entrenando LDA con {NUM_TOPICS} tópicos (passes={PASSES})...")
lda_model = entrenar_lda(
    corpus=corpus,
    diccionario=diccionario,
    num_topics=NUM_TOPICS,
    passes=PASSES
)
print("Modelo LDA entrenado.\n")

# 6) Evaluar modelo (perplejidad y coherencia)
print("Evaluando modelo LDA...")
perplejidad, coherencia = evaluar_lda(
    lda_model=lda_model,
    corpus=corpus,
    textos_tokenizados=textos_finales,
    diccionario=diccionario
)
print(f"► Perplejidad: {perplejidad:.4f}")
print(f"► Coherencia (c_v): {coherencia:.4f}\n")

# 7) Mostrar top-10 palabras de cada tópico
print("Términos más representativos por tópico:")
for tid in range(NUM_TOPICS):
    términos = lda_model.show_topic(tid, topn=10)
    lista_palabras = ", ".join([palabra for palabra, _ in términos])
    print(f"  Tópico {tid}: {lista_palabras}")
print()

# 8) Asignar tópico dominante a cada documento
print("Asignando tópico dominante a cada documento...")
df['bow'] = corpus
df['topic_principal'] = asignar_topic_principal(lda_model, corpus)

Cargados 1292 registros desde 'gestionspider5.json'.

Preprocesando descripciones (tokenización + stemming)...


Progreso preprocesado: 100%|██████████| 1292/1292 [00:04<00:00, 262.61it/s]



Detectando bigramas en el corpus...

Construyendo diccionario y corpus BoW...
Diccionario creado: 4690 tokens únicos.

Entrenando LDA con 7 tópicos (passes=10)...
Modelo LDA entrenado.

Evaluando modelo LDA...
► Perplejidad: -7.4595
► Coherencia (c_v): 0.3879

Términos más representativos por tópico:
  Tópico 0: unid, chin, arancel, pais, trump, estadounidens, president, dij, donald_trump, acuerd
  Tópico 1: millon, peru, pais, mayor, sector, inversion, export, crecimient, peruan, indic
  Tópico 2: trabaj, pued, empres, accion, pag, laboral, hac, fond, sol, millon
  Tópico 3: proyect, nacional, millon, congres, peru, inversion, pais, aprob, public, nuev
  Tópico 4: inform, public, nuev, cas, present, nacional, servici, proces, investig, oper
  Tópico 5: president, gobiern, ministr, public, peru, trabaj, miner, pais, congres, segur
  Tópico 6: empres, oper, peru, nuev, marc, merc, proyect, millon, inversion, peruan

Asignando tópico dominante a cada documento...


### Identificación del tópico a 5 noticias

In [9]:
# 9) Mostrar las primeras filas con su categoría original y tópico asignado
print("\nEjemplo de asignación de tópicos:")
print(df[['description', 'category', 'topic_principal']].head(5))


Ejemplo de asignación de tópicos:
                                         description             category  \
0  El expresidente de Estados Unidos, Joe Biden, ...                 EEUU   
1  En un contexto donde cada vez más personas opt...             Economía   
2  En abril, el sueldo requerido en el Perú se ub...  Management & Empleo   
3  El Gobierno aprobó la Política Nacional de Pro...                 Perú   
4  Este domingo 1 de junio comenzará a operar ofi...                 Perú   

   topic_principal  
0                0  
1                2  
2                1  
3                4  
4                4  


In [9]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS as GENSIM_STOP
from gensim import corpora
from gensim.models import LdaModel, CoherenceModel, Phrases, TfidfModel
from gensim.models.phrases import Phraser
from tqdm import tqdm

# -------------------------------------------------
# 1. Descargas NLTK (solo la primera vez)
# -------------------------------------------------
nltk.download('stopwords', quiet=True)

# -------------------------------------------------
# 2. Definir stopwords en español y stemmer
# -------------------------------------------------
nltk_stop = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')


# -------------------------------------------------
# 3. Función de preprocesado (tokenización + stemming)
# -------------------------------------------------
def preprocess(texto: str) -> list[str]:
    """
    1) Convierte a minúsculas y tokeniza quitando puntuación y acentos (deacc=True).
    2) Filtra tokens de longitud <= 3.
    3) Elimina stopwords de gensim y NLTK.
    4) Aplica stemming en español.
    Devuelve la lista de stems resultantes.
    """
    tokens = simple_preprocess(texto, deacc=True)
    tokens = [t for t in tokens if len(t) > 3 and t not in GENSIM_STOP and t not in nltk_stop]
    stems = [stemmer.stem(t) for t in tokens]
    return stems


# -------------------------------------------------
# 4. Generación de bigramas (opcional)
# -------------------------------------------------
def aplicar_bigramas(documentos: list[list[str]],
                     min_count: int = 5,
                     threshold: float = 100.0) -> tuple[list[list[str]], Phraser]:
    """
    1) Entrena un modelo de bigramas sobre los documentos tokenizados.
    2) Devuelve los documentos enriquecidos con bigramas (uni_bi) y el modelo Phraser.
    """
    bigram = Phrases(documentos, min_count=min_count, threshold=threshold, progress_per=10000)
    bigram_mod = Phraser(bigram)
    documentos_bi = [bigram_mod[doc] for doc in documentos]
    return documentos_bi, bigram_mod


# -------------------------------------------------
# 5. Construcción de diccionario y corpus BoW
# -------------------------------------------------
def construir_diccionario_corpus(textos_tokenizados: list[list[str]],
                                 no_below: int = 5,
                                 no_above: float = 0.5) -> tuple[corpora.Dictionary, list]:
    """
    1) Crea un Dictionary a partir de la lista de documentos (lista de tokens).
    2) Filtra palabras muy raras (no_below) o demasiado frecuentes (no_above).
    3) Construye el corpus en formato BoW (lista de tuplas (id_token, frecuencia)).
    Devuelve el diccionario filtrado y el corpus BoW.
    """
    diccionario = corpora.Dictionary(textos_tokenizados)
    diccionario.filter_extremes(no_below=no_below, no_above=no_above)
    corpus_bow = [diccionario.doc2bow(doc) for doc in textos_tokenizados]
    return diccionario, corpus_bow


# -------------------------------------------------
# 6. Entrenamiento y evaluación del modelo LDA
# -------------------------------------------------
def entrenar_lda(corpus: list, diccionario: corpora.Dictionary,
                 num_topics: int = 7,
                 passes: int = 10,
                 random_state: int = 42,
                 alpha: str = 'auto') -> LdaModel:
    """
    Entrena un modelo LDA con los parámetros indicados y devuelve el objeto LdaModel.
    """
    lda_model = LdaModel(
        corpus=corpus,
        id2word=diccionario,
        num_topics=num_topics,
        random_state=random_state,
        passes=passes,
        alpha=alpha,
        per_word_topics=True
    )
    return lda_model


def evaluar_lda(lda_model: LdaModel,
                corpus: list,
                textos_tokenizados: list[list[str]],
                diccionario: corpora.Dictionary) -> tuple[float, float]:
    """
    1) Calcula la perplejidad del modelo sobre el corpus.
    2) Calcula la coherencia (c_v) usando CoherenceModel.
    Devuelve (perplejidad, coherencia).
    """
    perplejidad = lda_model.log_perplexity(corpus)
    coh_model = CoherenceModel(model=lda_model,
                               texts=textos_tokenizados,
                               dictionary=diccionario,
                               coherence='c_v')
    coherencia = coh_model.get_coherence()
    return perplejidad, coherencia


# -------------------------------------------------
# 7. Asignación de tópico dominante a cada documento
# -------------------------------------------------
def asignar_topic_principal(lda_model: LdaModel, corpus: list) -> list[int]:
    """
    Para cada documento en formato BoW o TF-IDF, obtiene la lista de tópicos
    con sus probabilidades y retorna el ID del tópico con mayor probabilidad.
    """
    topicos_principales = []
    for doc_vector in corpus:
        distribucion = lda_model.get_document_topics(doc_vector, minimum_probability=0.0)
        topico_dom = max(distribucion, key=lambda x: x[1])[0]
        topicos_principales.append(topico_dom)
    return topicos_principales


# -------------------------------------------------
# 8. Función principal: pipeline completo
# -------------------------------------------------
def main():
    # Parámetros ajustables
    RUTA_CSV = "peru21news2.csv"      # Nombre del CSV a leer
    NUM_TOPICS = 7                    # Número de tópicos para LDA
    PASSES = 10                       # Número de pasadas en LDA
    NO_BELOW = 5                      # Mínimo de documentos para que un token sobreviva
    NO_ABOVE = 0.5                    # Máximo porcentaje de documentos para que un token sobreviva
    BIGRAM = True                     # Si generar bigramas (True/False)
    BIGRAM_MIN_COUNT = 5              # Umbral mínimo de apariciones conjuntas para bigramas
    BIGRAM_THRESHOLD = 100.0          # Umbral de formación de bigramas

    # 1) Leer el CSV (solo columnas 'summit' y 'category')
    df = pd.read_csv(RUTA_CSV, delimiter='|', encoding='utf-8', usecols=['summit', 'category'])
    print(f"Cargados {len(df)} registros desde '{RUTA_CSV}'.\n")

    # 2) Preprocesar los textos (tokenización + stemming) con tqdm
    textos_raw = df['summit'].astype(str).tolist()
    documentos_tokenizados = []
    print("1) Preprocesando textos (tokenización + stemming)...")
    for doc in tqdm(textos_raw, desc="Preprocesado"):
        documentos_tokenizados.append(preprocess(doc))

    # 3) (Opcional) Generar e incluir bigramas
    if BIGRAM:
        print("\n2) Detectando bigramas en el corpus...")
        documentos_bi, modelo_bigram = aplicar_bigramas(
            documentos_tokenizados,
            min_count=BIGRAM_MIN_COUNT,
            threshold=BIGRAM_THRESHOLD
        )
        textos_finales = documentos_bi
    else:
        textos_finales = documentos_tokenizados

    # 4) Construir diccionario y corpus BoW
    print("\n3) Construyendo diccionario y corpus BoW...")
    diccionario, corpus_bow = construir_diccionario_corpus(
        textos_tokenizados=textos_finales,
        no_below=NO_BELOW,
        no_above=NO_ABOVE
    )
    print(f"   - Diccionario creado con {len(diccionario)} tokens únicos.\n")

    # -------------------------------------------------
    # 5) CÁLCULO DE TF-IDF
    # -------------------------------------------------
    print("4) Calculando modelo TF-IDF sobre el corpus BoW...")
    tfidf_model = TfidfModel(corpus_bow, dictionary=diccionario)
    corpus_tfidf = tfidf_model[corpus_bow]

    # 5.1) EXTRAER PALABRAS MÁS RELEVANTES (suma de TF-IDF por token)
    print("   → Extrayendo las palabras más relevantes (sumando TF-IDF por token)...")
    # Creamos un vector global que acumule TF-IDF de cada token sobre todos los docuementos
    tfidf_global = {}
    for doc in corpus_tfidf:
        for token_id, tfidf_val in doc:
            if token_id not in tfidf_global:
                tfidf_global[token_id] = tfidf_val
            else:
                tfidf_global[token_id] += tfidf_val

    # Ordenamos todos los tokens por peso TF-IDF acumulado descendente
    tokens_ordenados = sorted(tfidf_global.items(), key=lambda x: x[1], reverse=True)
    TOP_TFIDF = 20
    print(f"\n   Top {TOP_TFIDF} tokens más relevantes (TF-IDF acumulado):")
    for idx, (token_id, acumulado) in enumerate(tokens_ordenados[:TOP_TFIDF], start=1):
        palabra = diccionario[token_id]
        print(f"     {idx:>2}. {palabra:<20}  —  TF-IDF acumulado: {acumulado:.4f}")
    print()

    # -------------------------------------------------
    # 6) ENTRENAR LDA sobre CORPUS TF-IDF
    # -------------------------------------------------
    print("5) Entrenando LDA (7 tópicos) sobre el corpus TF-IDF...")
    lda_model = entrenar_lda(
        corpus=list(corpus_tfidf),   # IMPORTANTE: usamos corpus_tfidf en lugar de corpus_bow
        diccionario=diccionario,
        num_topics=NUM_TOPICS,
        passes=PASSES
    )
    print("   ► LDA entrenado.\n")

    # 6.1) Evaluar modelo (perplejidad y coherencia)
    print("6) Evaluando modelo LDA...")
    perplejidad, coherencia = evaluar_lda(
        lda_model=lda_model,
        corpus=list(corpus_tfidf),
        textos_tokenizados=textos_finales,
        diccionario=diccionario
    )
    print(f"   ► Perplejidad: {perplejidad:.4f}")
    print(f"   ► Coherencia (c_v): {coherencia:.4f}\n")

    # 6.2) Mostrar top-10 palabras de cada tópico
    print("7) Términos más representativos por tópico (Top 10):")
    for tid in range(NUM_TOPICS):
        términos = lda_model.show_topic(tid, topn=10)
        términos_str = ", ".join([pal for pal, _ in términos])
        print(f"   Tópico {tid}: {términos_str}")
    print()

    # -------------------------------------------------
    # 7) ASIGNAR TÓPICO DOMINANTE A CADA DOCUMENTO
    # -------------------------------------------------
    print("8) Asignando tópico dominante a cada noticia...")
    df['bow'] = corpus_bow
    # NOTA: La función `asignar_topic_principal` funciona con corpus en BoW o en TF-IDF
    df['topic_principal'] = asignar_topic_principal(lda_model, list(corpus_tfidf))
    print("   → Asignación completada.\n")

    # -------------------------------------------------
    # 8) MOSTRAR RESULTADOS PARA 3 NOTICIAS EJEMPLO
    # -------------------------------------------------
    print("9) Mostrando resultados de LDA para 3 noticias de ejemplo:\n")
    # Seleccionamos tres índices arbitrarios (por ejemplo 0, 10 y 20) o aleatorios
    ejemplos_idx = [0, 10, 20]
    for idx in ejemplos_idx:
        texto_original = df.loc[idx, 'summit']
        categoria = df.loc[idx, 'category']
        bow         = df.loc[idx, 'bow']
        tfidf_vec   = corpus_tfidf[idx]
        topico_dom  = df.loc[idx, 'topic_principal']

        # Obtenemos la distribución completa de tópicos para ese documento
        distribucion = lda_model.get_document_topics(tfidf_vec, minimum_probability=0.0)

        print(f"→ Noticia #{idx} (categoría = '{categoria}'):")
        print(f"   * Texto original (resumen): {texto_original[:100]}...")  # mostramos solo los primeros 100 caracteres
        print(f"   * Tópico dominante: {topico_dom}")
        print(f"   * Distribución completa de tópicos (id: probabilidad):")
        print(f"     {[(t, round(p, 4)) for t, p in distribucion]}\n")


if __name__ == "__main__":
    main()


Cargados 108 registros desde 'peru21news2.csv'.

1) Preprocesando textos (tokenización + stemming)...


Preprocesado: 100%|██████████| 108/108 [00:00<00:00, 594.07it/s]
2025-05-31 21:27:10,030 : INFO : collecting all words and their counts
2025-05-31 21:27:10,032 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2025-05-31 21:27:10,038 : INFO : collected 2089 token types (unigram + bigrams) from a corpus of 1340 words and 108 sentences
2025-05-31 21:27:10,038 : INFO : merged Phrases<2089 vocab, min_count=5, threshold=100.0, max_vocab_size=40000000>
2025-05-31 21:27:10,038 : INFO : Phrases lifecycle event {'msg': 'built Phrases<2089 vocab, min_count=5, threshold=100.0, max_vocab_size=40000000> in 0.01s', 'datetime': '2025-05-31T21:27:10.038148', 'gensim': '4.3.3', 'python': '3.12.3 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:42:21) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-11-10.0.26100-SP0', 'event': 'created'}
2025-05-31 21:27:10,038 : INFO : exporting phrases from Phrases<2089 vocab, min_count=5, threshold=100.0, max_vocab_size=40000000>
2025-05-3


2) Detectando bigramas en el corpus...

3) Construyendo diccionario y corpus BoW...
   - Diccionario creado con 15 tokens únicos.

4) Calculando modelo TF-IDF sobre el corpus BoW...
   → Extrayendo las palabras más relevantes (sumando TF-IDF por token)...

   Top 20 tokens más relevantes (TF-IDF acumulado):
      1. peru                  —  TF-IDF acumulado: 6.1172
      2. peruan                —  TF-IDF acumulado: 5.0400
      3. aere                  —  TF-IDF acumulado: 4.9935
      4. oper                  —  TF-IDF acumulado: 4.9335
      5. nuev                  —  TF-IDF acumulado: 4.6406
      6. millon                —  TF-IDF acumulado: 4.2167
      7. carg                  —  TF-IDF acumulado: 4.1775
      8. public                —  TF-IDF acumulado: 4.1745
      9. anos                  —  TF-IDF acumulado: 4.1732
     10. sol                   —  TF-IDF acumulado: 4.1732
     11. pais                  —  TF-IDF acumulado: 4.0670
     12. muert                 —  TF-IDF 

2025-05-31 21:27:10,245 : INFO : topic #6 (0.136): 0.334*"muert" + 0.204*"peruan" + 0.148*"aere" + 0.026*"oper" + 0.026*"peru" + 0.026*"segur" + 0.026*"carg" + 0.026*"nuev" + 0.026*"sol" + 0.026*"anos"
2025-05-31 21:27:10,280 : INFO : topic #1 (0.137): 0.310*"anos" + 0.166*"peru" + 0.108*"segur" + 0.103*"oper" + 0.103*"public" + 0.021*"peruan" + 0.021*"carg" + 0.021*"aere" + 0.021*"sol" + 0.021*"millon"
2025-05-31 21:27:10,280 : INFO : topic #2 (0.145): 0.258*"aere" + 0.176*"oper" + 0.176*"segur" + 0.094*"anos" + 0.094*"nuev" + 0.094*"peruan" + 0.012*"peru" + 0.012*"carg" + 0.012*"pais" + 0.012*"millon"
2025-05-31 21:27:10,280 : INFO : topic #0 (0.148): 0.190*"pais" + 0.172*"nuev" + 0.132*"seguidor" + 0.112*"peruan" + 0.092*"public" + 0.069*"segur" + 0.052*"anos" + 0.051*"muert" + 0.044*"carg" + 0.042*"nacional"
2025-05-31 21:27:10,280 : INFO : topic #3 (0.148): 0.209*"nacional" + 0.162*"carg" + 0.149*"oper" + 0.109*"public" + 0.087*"seguidor" + 0.068*"millon" + 0.062*"peruan" + 0.048*

   ► LDA entrenado.

6) Evaluando modelo LDA...


2025-05-31 21:27:17,838 : INFO : 11 accumulators retrieved from output queue
2025-05-31 21:27:17,849 : INFO : accumulated word occurrence stats for 108 virtual documents


   ► Perplejidad: -3.8408
   ► Coherencia (c_v): 0.4622

7) Términos más representativos por tópico (Top 10):
   Tópico 0: nuev, pais, public, seguidor, peruan, segur, anos, muert, carg, nacional
   Tópico 1: anos, peru, segur, public, oper, pais, muert, peruan, sol, carg
   Tópico 2: oper, aere, segur, nuev, peruan, anos, public, nacional, peru, muert
   Tópico 3: carg, nacional, seguidor, public, oper, peruan, millon, peru, segur, nuev
   Tópico 4: peru, sol, pais, peruan, aere, anos, millon, nuev, nacional, segur
   Tópico 5: millon, sol, muert, public, peru, carg, peruan, nuev, oper, pais
   Tópico 6: peruan, muert, aere, nacional, nuev, pais, anos, sol, peru, oper

8) Asignando tópico dominante a cada noticia...
   → Asignación completada.

9) Mostrando resultados de LDA para 3 noticias de ejemplo:

→ Noticia #0 (categoría = 'Política'):
   * Texto original (resumen): El Registro Nacional de Identificación y Estado Civil y la Cancillería peruana coordinan e impulsan ...
   * Tópic